In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!unzip -q drive/'My Drive'/dataset-original.zip

In [0]:
!mkdir dataset-validation
!mkdir dataset-validation/cardboard
!mkdir dataset-validation/glass
!mkdir dataset-validation/paper
!mkdir dataset-validation/plastic
!mkdir dataset-validation/metal
!mkdir dataset-validation/trash
!ls
!ls dataset-original/trash

dataset-original  dataset-validation  drive  __MACOSX  sample_data
trash100.jpg  trash121.jpg  trash18.jpg  trash39.jpg  trash5.jpg   trash80.jpg
trash101.jpg  trash122.jpg  trash19.jpg  trash3.jpg   trash60.jpg  trash81.jpg
trash102.jpg  trash123.jpg  trash1.jpg	 trash40.jpg  trash61.jpg  trash82.jpg
trash103.jpg  trash124.jpg  trash20.jpg  trash41.jpg  trash62.jpg  trash83.jpg
trash104.jpg  trash125.jpg  trash21.jpg  trash42.jpg  trash63.jpg  trash84.jpg
trash105.jpg  trash126.jpg  trash22.jpg  trash43.jpg  trash64.jpg  trash85.jpg
trash106.jpg  trash127.jpg  trash23.jpg  trash44.jpg  trash65.jpg  trash86.jpg
trash107.jpg  trash128.jpg  trash24.jpg  trash45.jpg  trash66.jpg  trash87.jpg
trash108.jpg  trash129.jpg  trash25.jpg  trash46.jpg  trash67.jpg  trash88.jpg
trash109.jpg  trash12.jpg   trash26.jpg  trash47.jpg  trash68.jpg  trash89.jpg
trash10.jpg   trash130.jpg  trash27.jpg  trash48.jpg  trash69.jpg  trash8.jpg
trash110.jpg  trash131.jpg  trash28.jpg  trash49.jpg  trash6.jpg  

In [0]:
!shuf -n 101 -e dataset-original/glass/* | xargs -i mv {} dataset-validation/glass/
!shuf -n 144 -e dataset-original/paper/* | xargs -i mv {} dataset-validation/paper/
!shuf -n 53 -e dataset-original/cardboard/* | xargs -i mv {} dataset-validation/cardboard/
!shuf -n 82 -e dataset-original/plastic/* | xargs -i mv {} dataset-validation/plastic/
!shuf -n 60 -e dataset-original/metal/* | xargs -i mv {} dataset-validation/metal/
!shuf -n 37 -e dataset-original/trash/* | xargs -i mv {} dataset-validation/trash/

In [0]:
!ls dataset-original/glass | wc -l
!ls dataset-validation/glass | wc -l

400
101


In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import numpy as np

In [0]:
transformationsTrain = transforms.Compose([transforms.Resize(255), transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip(),
                                           transforms.RandomRotation(degrees = (90, -90)), transforms.ToTensor(), 
                                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transformationsValidation = transforms.Compose([transforms.Resize(255), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [0]:
train_set = datasets.ImageFolder("dataset-original", transform = transformationsTrain)
val_set = datasets.ImageFolder("dataset-validation", transform = transformationsValidation)

In [0]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=50, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size =50, shuffle=True)

In [0]:
model = torchvision.models.resnet34()
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [0]:
learningRate = 1e-2
epochs = 5
minibatches = 5

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), learningRate)

train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        pred = outputs.data.max(1)[1]
        matches = labels == pred
        accuracy = matches.float().mean()

        # print statistics
        running_loss += loss.item()
        if i % minibatches == minibatches - 1:    # print every n mini-batches
            print('[%d, %5d] loss: %.3f accuracy: %.3f itemLoss: %.3f' %
                  (epoch + 1, i + 1, running_loss / minibatches, accuracy, loss.item()))
            running_loss = 0.0

        train_losses.append(loss.item())
        train_accs.append(accuracy.item())
    
    model.eval()
    val_loss, correct = 0., 0
    total = 0
    with torch.no_grad():
      for data in val_loader:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = model(inputs)
        val_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the validation images: %d %%' % (
    100 * correct / total))

    val_loss /= len(val_loader)
    acc = correct / len(val_loader)

    val_losses.append(val_loss)
    val_accs.append(acc)

print('Finished Training')

[1,     5] loss: 4.889 accuracy: 0.240 itemLoss: 3.080
[1,    10] loss: 2.466 accuracy: 0.240 itemLoss: 2.243
[1,    15] loss: 2.034 accuracy: 0.240 itemLoss: 2.033
[1,    20] loss: 1.887 accuracy: 0.360 itemLoss: 1.641
[1,    25] loss: 1.772 accuracy: 0.240 itemLoss: 1.794
[1,    30] loss: 1.667 accuracy: 0.300 itemLoss: 1.711
[1,    35] loss: 1.638 accuracy: 0.400 itemLoss: 1.621
[1,    40] loss: 1.636 accuracy: 0.360 itemLoss: 1.789
Accuracy of the network on the validation images: 34 %
[2,     5] loss: 1.552 accuracy: 0.420 itemLoss: 1.485
[2,    10] loss: 1.483 accuracy: 0.600 itemLoss: 1.204
[2,    15] loss: 1.392 accuracy: 0.380 itemLoss: 1.388
[2,    20] loss: 1.473 accuracy: 0.420 itemLoss: 1.541
[2,    25] loss: 1.486 accuracy: 0.500 itemLoss: 1.256
[2,    30] loss: 1.360 accuracy: 0.340 itemLoss: 1.581
[2,    35] loss: 1.450 accuracy: 0.320 itemLoss: 1.447
[2,    40] loss: 1.481 accuracy: 0.400 itemLoss: 1.761
Accuracy of the network on the validation images: 33 %
[3,     5]

In [0]:
torch.save(model.state_dict(), "/content/drive/My Drive/modelNov26.pt")

In [0]:
# model = torchvision.models.resnet34()
# model.cuda()
# model.load_state_dict(torch.load("/content/drive/My Drive/modelNov25.pt"))

<All keys matched successfully>

In [0]:
model = torchvision.models.resnet34(pretrained=True)
model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), learningRate)

train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        pred = outputs.data.max(1)[1]
        matches = labels == pred
        accuracy = matches.float().mean()

        # print statistics
        running_loss += loss.item()
        if i % minibatches == minibatches - 1:    # print every n mini-batches
            print('[%d, %5d] loss: %.3f accuracy: %.3f itemLoss: %.3f' %
                  (epoch + 1, i + 1, running_loss / minibatches, accuracy, loss.item()))
            running_loss = 0.0

        train_losses.append(loss.item())
        train_accs.append(accuracy.item())
    
    model.eval()
    val_loss, correct = 0., 0
    total = 0
    with torch.no_grad():
      for data in val_loader:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = model(inputs)
        val_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the validation images: %d %%' % (
    100 * correct / total))

    val_loss /= len(val_loader)
    acc = correct / len(val_loader)

    val_losses.append(val_loss)
    val_accs.append(acc)

print('Finished Training')

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 160MB/s] 


[1,     5] loss: 7.152 accuracy: 0.080 itemLoss: 4.820
[1,    10] loss: 2.577 accuracy: 0.660 itemLoss: 1.441
[1,    15] loss: 1.525 accuracy: 0.580 itemLoss: 1.391
[1,    20] loss: 1.054 accuracy: 0.820 itemLoss: 0.753
[1,    25] loss: 1.001 accuracy: 0.700 itemLoss: 1.041
[1,    30] loss: 0.908 accuracy: 0.760 itemLoss: 0.837
[1,    35] loss: 0.727 accuracy: 0.780 itemLoss: 0.702
[1,    40] loss: 0.711 accuracy: 0.860 itemLoss: 0.462
Accuracy of the network on the validation images: 76 %
[2,     5] loss: 0.559 accuracy: 0.860 itemLoss: 0.442
[2,    10] loss: 0.592 accuracy: 0.820 itemLoss: 0.596
[2,    15] loss: 0.547 accuracy: 0.820 itemLoss: 0.466
[2,    20] loss: 0.571 accuracy: 0.840 itemLoss: 0.579
[2,    25] loss: 0.467 accuracy: 0.740 itemLoss: 0.753
[2,    30] loss: 0.478 accuracy: 0.840 itemLoss: 0.527
[2,    35] loss: 0.345 accuracy: 0.840 itemLoss: 0.405
[2,    40] loss: 0.456 accuracy: 0.820 itemLoss: 0.434
Accuracy of the network on the validation images: 87 %
[3,     5]

In [0]:
model = torchvision.models.resnet34(pretrained=True)
model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learningRate)

train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        pred = outputs.data.max(1)[1]
        matches = labels == pred
        accuracy = matches.float().mean()

        # print statistics
        running_loss += loss.item()
        if i % minibatches == minibatches - 1:    # print every n mini-batches
            print('[%d, %5d] loss: %.3f accuracy: %.3f itemLoss: %.3f' %
                  (epoch + 1, i + 1, running_loss / minibatches, accuracy, loss.item()))
            running_loss = 0.0

        train_losses.append(loss.item())
        train_accs.append(accuracy.item())
    
    model.eval()
    val_loss, correct = 0., 0
    total = 0
    with torch.no_grad():
      for data in val_loader:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = model(inputs)
        val_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the validation images: %d %%' % (
    100 * correct / total))

    val_loss /= len(val_loader)
    acc = correct / len(val_loader)

    val_losses.append(val_loss)
    val_accs.append(acc)

print('Finished Training')

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 106MB/s]


[1,     5] loss: 4.935 accuracy: 0.180 itemLoss: 2.417
[1,    10] loss: 2.181 accuracy: 0.240 itemLoss: 2.429
[1,    15] loss: 1.820 accuracy: 0.300 itemLoss: 1.686
[1,    20] loss: 1.784 accuracy: 0.280 itemLoss: 2.778
[1,    25] loss: 1.981 accuracy: 0.320 itemLoss: 1.869
[1,    30] loss: 1.950 accuracy: 0.340 itemLoss: 2.015
[1,    35] loss: 1.655 accuracy: 0.380 itemLoss: 1.600
[1,    40] loss: 1.553 accuracy: 0.360 itemLoss: 1.349
Accuracy of the network on the validation images: 28 %
[2,     5] loss: 1.950 accuracy: 0.340 itemLoss: 1.573
[2,    10] loss: 1.817 accuracy: 0.400 itemLoss: 1.928
[2,    15] loss: 1.660 accuracy: 0.360 itemLoss: 1.630
[2,    20] loss: 1.515 accuracy: 0.360 itemLoss: 1.470
[2,    25] loss: 1.370 accuracy: 0.500 itemLoss: 1.210
[2,    30] loss: 1.491 accuracy: 0.400 itemLoss: 1.539
[2,    35] loss: 1.620 accuracy: 0.320 itemLoss: 1.937
[2,    40] loss: 1.419 accuracy: 0.420 itemLoss: 1.529
Accuracy of the network on the validation images: 34 %
[3,     5]

In [0]:
model = torchvision.models.resnet34(pretrained=True)
model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        pred = outputs.data.max(1)[1]
        matches = labels == pred
        accuracy = matches.float().mean()

        # print statistics
        running_loss += loss.item()
        if i % minibatches == minibatches - 1:    # print every n mini-batches
            print('[%d, %5d] loss: %.3f accuracy: %.3f itemLoss: %.3f' %
                  (epoch + 1, i + 1, running_loss / minibatches, accuracy, loss.item()))
            running_loss = 0.0

        train_losses.append(loss.item())
        train_accs.append(accuracy.item())
    
    model.eval()
    val_loss, correct = 0., 0
    total = 0
    with torch.no_grad():
      for data in val_loader:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = model(inputs)
        val_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the validation images: %d %%' % (
    100 * correct / total))

    val_loss /= len(val_loader)
    acc = correct / len(val_loader)

    val_losses.append(val_loss)
    val_accs.append(acc)

print('Finished Training')

[1,     5] loss: 4.520 accuracy: 0.740 itemLoss: 1.642
[1,    10] loss: 1.870 accuracy: 0.440 itemLoss: 2.105
[1,    15] loss: 1.459 accuracy: 0.480 itemLoss: 1.806
[1,    20] loss: 1.226 accuracy: 0.640 itemLoss: 1.219
[1,    25] loss: 1.096 accuracy: 0.660 itemLoss: 1.006
[1,    30] loss: 0.903 accuracy: 0.640 itemLoss: 1.008
[1,    35] loss: 1.104 accuracy: 0.580 itemLoss: 1.161
[1,    40] loss: 1.012 accuracy: 0.620 itemLoss: 1.240
Accuracy of the network on the validation images: 62 %
[2,     5] loss: 0.974 accuracy: 0.600 itemLoss: 1.107
[2,    10] loss: 0.836 accuracy: 0.680 itemLoss: 0.822
[2,    15] loss: 0.835 accuracy: 0.760 itemLoss: 0.712
[2,    20] loss: 0.771 accuracy: 0.660 itemLoss: 0.701
[2,    25] loss: 0.713 accuracy: 0.700 itemLoss: 0.739
[2,    30] loss: 0.807 accuracy: 0.740 itemLoss: 0.663
[2,    35] loss: 0.633 accuracy: 0.760 itemLoss: 0.701
[2,    40] loss: 0.661 accuracy: 0.760 itemLoss: 0.569
Accuracy of the network on the validation images: 73 %
[3,     5]

In [0]:
!pip install adabound

import adabound

In [0]:
model = torchvision.models.resnet34(pretrained=True)
model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = adabound.AdaBound(model.parameters(), lr=1e-3, final_lr=0.1)

train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        pred = outputs.data.max(1)[1]
        matches = labels == pred
        accuracy = matches.float().mean()

        # print statistics
        running_loss += loss.item()
        if i % minibatches == minibatches - 1:    # print every n mini-batches
            print('[%d, %5d] loss: %.3f accuracy: %.3f itemLoss: %.3f' %
                  (epoch + 1, i + 1, running_loss / minibatches, accuracy, loss.item()))
            running_loss = 0.0

        train_losses.append(loss.item())
        train_accs.append(accuracy.item())
    
    model.eval()
    val_loss, correct = 0., 0
    total = 0
    with torch.no_grad():
      for data in val_loader:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = model(inputs)
        val_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the validation images: %d %%' % (
    100 * correct / total))

    val_loss /= len(val_loader)
    acc = correct / len(val_loader)

    val_losses.append(val_loss)
    val_accs.append(acc)

print('Finished Training')

[1,     5] loss: 4.873 accuracy: 0.500 itemLoss: 3.133
[1,    10] loss: 2.033 accuracy: 0.620 itemLoss: 1.243
[1,    15] loss: 1.351 accuracy: 0.600 itemLoss: 1.349
[1,    20] loss: 1.074 accuracy: 0.660 itemLoss: 1.006
[1,    25] loss: 1.097 accuracy: 0.620 itemLoss: 1.076
[1,    30] loss: 1.081 accuracy: 0.640 itemLoss: 0.882
[1,    35] loss: 0.842 accuracy: 0.740 itemLoss: 0.726
[1,    40] loss: 0.859 accuracy: 0.560 itemLoss: 0.949
Accuracy of the network on the validation images: 57 %
[2,     5] loss: 0.744 accuracy: 0.820 itemLoss: 0.540
[2,    10] loss: 0.723 accuracy: 0.720 itemLoss: 0.657
[2,    15] loss: 0.792 accuracy: 0.680 itemLoss: 0.742
[2,    20] loss: 0.683 accuracy: 0.700 itemLoss: 0.933
[2,    25] loss: 0.805 accuracy: 0.640 itemLoss: 1.108
[2,    30] loss: 0.776 accuracy: 0.680 itemLoss: 0.792
[2,    35] loss: 0.629 accuracy: 0.800 itemLoss: 0.685
[2,    40] loss: 0.726 accuracy: 0.640 itemLoss: 0.729
Accuracy of the network on the validation images: 70 %
[3,     5]

In [17]:
model = torchvision.models.resnet34(pretrained=True)
model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        pred = outputs.data.max(1)[1]
        matches = labels == pred
        accuracy = matches.float().mean()

        # print statistics
        running_loss += loss.item()
        if i % minibatches == minibatches - 1:    # print every n mini-batches
            print('[%d, %5d] loss: %.3f accuracy: %.3f itemLoss: %.3f' %
                  (epoch + 1, i + 1, running_loss / minibatches, accuracy, loss.item()))
            running_loss = 0.0

        train_losses.append(loss.item())
        train_accs.append(accuracy.item())
    
    model.eval()
    val_loss, correct = 0., 0
    total = 0
    with torch.no_grad():
      for data in val_loader:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = model(inputs)
        val_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the validation images: %d %%' % (
    100 * correct / total))

    val_loss /= len(val_loader)
    acc = correct / len(val_loader)

    val_losses.append(val_loss)
    val_accs.append(acc)

print('Finished Training')

[1,     5] loss: 6.957 accuracy: 0.280 itemLoss: 4.763
[1,    10] loss: 3.008 accuracy: 0.440 itemLoss: 2.231
[1,    15] loss: 1.452 accuracy: 0.640 itemLoss: 1.265
[1,    20] loss: 0.949 accuracy: 0.700 itemLoss: 1.151
[1,    25] loss: 0.858 accuracy: 0.740 itemLoss: 0.869
[1,    30] loss: 0.840 accuracy: 0.740 itemLoss: 0.902
[1,    35] loss: 0.698 accuracy: 0.820 itemLoss: 0.599
[1,    40] loss: 0.622 accuracy: 0.680 itemLoss: 0.685
Accuracy of the network on the validation images: 77 %
[2,     5] loss: 0.521 accuracy: 0.920 itemLoss: 0.335
[2,    10] loss: 0.548 accuracy: 0.760 itemLoss: 0.775
[2,    15] loss: 0.532 accuracy: 0.720 itemLoss: 0.693
[2,    20] loss: 0.508 accuracy: 0.840 itemLoss: 0.421
[2,    25] loss: 0.527 accuracy: 0.860 itemLoss: 0.438
[2,    30] loss: 0.479 accuracy: 0.780 itemLoss: 0.592
[2,    35] loss: 0.370 accuracy: 0.800 itemLoss: 0.427
[2,    40] loss: 0.395 accuracy: 0.860 itemLoss: 0.400
Accuracy of the network on the validation images: 87 %
[3,     5]